# 최소 비용 최대 유량 (Minimum Cost Maximum Flow)

`-` 최대 유량을 흘리되 비용을 최소로 하자

## 열혈강호 5

- 문제 출처: [백준 11408번](https://www.acmicpc.net/problem/11408)

`-` 간선에 비용이 추가됐다

`-` 비용이 없고 유량과 용량만 있는 그래프에서 최대 유량을 찾을 땐 BFS를 통해 증가 경로를 찾았다

`-` 여기서는 최소 비용인 증가 경로를 찾기 위해 최단 거리 탐색 알고리즘을 사용할 것이다

`-` 단, 역방향 간선의 경우 유량이 음수일 수 있으므로 음수 간선이 존재해도 잘 동작하는 Bellman-Ford 알고리즘을 사용하자

`-` Bellman-Ford 알고리즘을 개량한 SPFA(Shortest Path Faster Algorithm)를 사용할 것이다

`-` Bellman-Ford 알고리즘에선 매 반복문마다 모든 간선을 순회하며 거리 갱신을 시도했지만 SPFA에선 거리가 갱신된 노드와 연결된 간선만 고려할 것이다

`-` 이러한 방식의 알고리즘을 SSP(Successive Shortest Path) 알고리즘이라 한다

`-` SPFA의 시간 복잡도는 최악의 경우 $O(VE)$이므로 SSP 알고리즘의 시간 복잡도는 $O(VEf)$이다 (단, $f$는 최대 유량)

`-` 매 증가 경로마다 최소 비용으로 구했기에 최대 유량을 흘릴 때의 비용 또한 최소 비용이다 (그렇지 않다면 모순이다)

`-` 참고: https://cp-algorithms.com/graph/min_cost_flow.html

`-` 참고: https://codeforces.com/blog/entry/105330

`-` 이 문제에선 최대 유량이 강호네 회사에서 할 수 있는 일의 개수의 최댓값이고 최소 비용이 일을 최대로 할 때 강호가 지급해야 하는 월급의 최솟값이다

In [28]:
from collections import defaultdict, deque


def ssp(graph, weight, source, sink, flow, capacity, num_nodes):
    max_flow = 0
    min_cost = 0
    exists_augmenting_path = True
    while exists_augmenting_path:
        predecessors, distances = spfa(graph, weight, source, flow, capacity, num_nodes)
        exists_augmenting_path = predecessors[sink] != NONE
        if not exists_augmenting_path:
            break
        augmenting_path = track_augmenting_path(predecessors, sink)
        bottleneck = compute_bottleneck(augmenting_path, flow, capacity)
        update_flow(augmenting_path, flow, bottleneck)
        cost = distances[sink]
        max_flow += bottleneck
        min_cost += cost
    return max_flow, min_cost


def spfa(graph, weight, source, flow, capacity, num_nodes):
    queue = deque([source])
    distances = [INF] * num_nodes
    distances[source] = 0
    predecessors = [NONE] * num_nodes
    in_queue = [False] * num_nodes
    in_queue[source] = True
    while queue:
        u = queue.popleft()
        dist_u = distances[u]
        in_queue[u] = False
        for v in graph[u]:
            w = weight[u][v]
            residual_capacity = capacity[u][v] - flow[u][v]
            dist_v = distances[v]
            if residual_capacity <= 0 or dist_v <= dist_u + w:
                continue
            distances[v] = dist_u + w
            predecessors[v] = u
            if in_queue[v]:
                continue
            in_queue[v] = True
            queue.append(v)
    return predecessors, distances


def track_augmenting_path(predecessors, sink):
    augmenting_path = []
    node = sink
    while node != NONE:
        augmenting_path.append(node)
        node = predecessors[node]
    augmenting_path.reverse()
    return augmenting_path


def compute_bottleneck(augmenting_path, flow, capacity):
    bottleneck = INF
    for i in range(len(augmenting_path) - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1]
        bottleneck = min(capacity[u][v] - flow[u][v], bottleneck)
    return bottleneck


def update_flow(augmenting_path, flow, bottleneck):
    for i in range(len(augmenting_path) - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1]
        flow[u][v] += bottleneck
        flow[v][u] -= bottleneck


def solution():
    global INF, NONE
    N, M = map(int, input().split())
    INF = float("inf")
    NONE = -1
    source = 0
    sink = N + M + 1
    num_nodes = N + M + 2
    flow = [[0] * num_nodes for _ in range(num_nodes)]
    capacity = [[1] * (num_nodes) for _ in range(num_nodes)]
    graph = defaultdict(list)
    graph[source] = list(range(1, N + 1))
    weight = [[0] * num_nodes for _ in range(num_nodes)]
    for work in range(N + 1, N + M + 1):
        graph[work].append(sink)
    for employee in range(1, N + 1):
        data = list(map(int, input().split()))
        n = data[0]
        for i in range(n):
            work = data[2 * i + 1] + N
            salary = data[2 * (i + 1)]
            graph[employee].append(work)
            graph[work].append(employee)
            capacity[work][employee] = 0
            weight[employee][work] = salary
            weight[work][employee] = -salary
    max_flow, min_cost = ssp(graph, weight, source, sink, flow, capacity, num_nodes)
    print(max_flow)
    print(min_cost)


solution()

# input
# 5 5
# 2 1 3 2 2
# 1 1 5
# 2 2 1 3 7
# 3 3 9 4 9 5 9
# 1 1 0

 5 5
 2 1 3 2 2
 1 1 5
 2 2 1 3 7
 3 3 9 4 9 5 9
 1 1 0


4
18


`-` spfa 구현할 때 큐에 넣으면 빼먹지 말고 큐에 들어있다고 표시해줘야 한다

`-` 처음엔 weight 행렬을 defaultdict으로 구현했었다

`-` 기본 값은 $0$으로 주었는데 자료 구조 자체가 느린 건지 아니면 부작용이 있었던 것인지 $2\%$도 못 가고 시간 초과가 계속 발생했다

`-` 이중 리스트로 구현하니까 바로 맞혔다

## 열혈강호 6

- 문제 출처: [백준 11409번](https://www.acmicpc.net/problem/11409)

`-` 일을 최대한 많이 시키되 강호가 지불해야 하는 월급도 최대로 해야 한다

`-` 최대 비용 최대 유량 문제(?)인데 강호가 지불하는 월급을 음수로 설정하면 그만이다

`-` 파이썬에서 힙을 사용할 때 최소 힙만 지원해서 최대 힙으로 사용하고 싶을 땐 원소에 $-1$을 곱해서 사용했었다

`-` 이것처럼 정방향 간선의 비용에 $-1$을 곱해주면 된다 (역방향 간선의 비용은 정방향 간선에 $-1$을 곱하면 됨)

`-` 최대 유량과 그 때의 최소 비용을 계산했으면 최소 비용에 다시 $-1$을 곱해주면 강호가 지불하는 월급의 최댓값을 알 수 있다

`-` 참고로 유량을 흘리다보면 음수 사이클이 생기지 않을까 걱정할 수 있다

`-` 만약 그래프에 없던 음수 사이클이 새로 생겼다면 유량을 흘려주고 역방향 간선이 활성화 되면서 생겼다는 의미이다

`-` 최소 비용 경로에 존재하는 간선에 대해 역방향 간선이 생기게 된다

`-` 해당 역방향 간선을 따라 음수 사이클이 생기는 것을 고려할 수 있는데 최소 비용인지라 사이클 내의 정방향 간선 쪽 비용과 합하면 $0$보다 클 수 밖에 없다

`-` 따라서 처음 그래프에 음수 사이클이 없다면 새롭게 음수 사이클이 생기지 않으니 걱정하지 않아도 된다

In [33]:
from collections import defaultdict, deque


def ssp(graph, weight, source, sink, flow, capacity, num_nodes):
    max_flow = 0
    min_cost = 0
    exists_augmenting_path = True
    while exists_augmenting_path:
        predecessors, distances = spfa(graph, weight, source, flow, capacity, num_nodes)
        exists_augmenting_path = predecessors[sink] != NONE
        if not exists_augmenting_path:
            break
        augmenting_path = track_augmenting_path(predecessors, sink)
        bottleneck = compute_bottleneck(augmenting_path, flow, capacity)
        update_flow(augmenting_path, flow, bottleneck)
        cost = distances[sink]
        max_flow += bottleneck
        min_cost += cost
    return max_flow, min_cost


def spfa(graph, weight, source, flow, capacity, num_nodes):
    queue = deque([source])
    distances = [INF] * num_nodes
    distances[source] = 0
    predecessors = [NONE] * num_nodes
    in_queue = [False] * num_nodes
    in_queue[source] = True
    while queue:
        u = queue.popleft()
        in_queue[u] = False
        dist_u = distances[u]
        for v in graph[u]:
            w = weight[u][v]
            dist_v = distances[v]
            residual_capacity = capacity[u][v] - flow[u][v]
            if residual_capacity <= 0 or dist_v <= dist_u + w:
                continue
            distances[v] = dist_u + w
            predecessors[v] = u
            if in_queue[v]:
                continue
            in_queue[v] = True
            queue.append(v)
    return predecessors, distances


def track_augmenting_path(predecessors, sink):
    augmenting_path = []
    node = sink
    while node != NONE:
        augmenting_path.append(node)
        node = predecessors[node]
    augmenting_path.reverse()
    return augmenting_path


def compute_bottleneck(augmenting_path, flow, capacity):
    bottleneck = INF
    for i in range(len(augmenting_path) - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1]
        bottleneck = min(capacity[u][v] - flow[u][v], bottleneck)
    return bottleneck


def update_flow(augmenting_path, flow, bottleneck):
    for i in range(len(augmenting_path) - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1]
        flow[u][v] += bottleneck
        flow[v][u] -= bottleneck


def solution():
    global INF, NONE
    N, M = map(int, input().split())
    INF = float("inf")
    NONE = -1
    source = 0
    sink = N + M + 1
    num_nodes = N + M + 2
    flow = [[0] * num_nodes for _ in range(num_nodes)]
    capacity = [[1] * (num_nodes) for _ in range(num_nodes)]
    graph = defaultdict(list)
    graph[source] = list(range(1, N + 1))
    weight = [[0] * num_nodes for _ in range(num_nodes)]
    for work in range(N + 1, N + M + 1):
        graph[work].append(sink)
    for employee in range(1, N + 1):
        data = list(map(int, input().split()))
        n = data[0]
        for i in range(n):
            work = data[2 * i + 1] + N
            salary = data[2 * (i + 1)]
            graph[employee].append(work)
            graph[work].append(employee)
            capacity[work][employee] = 0
            weight[employee][work] = -salary
            weight[work][employee] = salary
    max_flow, min_cost = ssp(graph, weight, source, sink, flow, capacity, num_nodes)
    max_cost = -min_cost
    print(max_flow)
    print(max_cost)


solution()

# input
# 5 5
# 2 1 3 2 2
# 1 1 5
# 2 2 1 3 7
# 3 3 9 4 9 5 9
# 1 1 0

 5 5
 2 1 3 2 2
 1 1 5
 2 2 1 3 7
 3 3 9 4 9 5 9
 1 1 0


4
23


## 책 구매하기

- 문제 출처: [백준 11405번](https://www.acmicpc.net/problem/11405)

`-` MCMF 기본 문제이다

`-` 그래프 설계만 정확히 하면 된다

`-` 소스와 사람 노드를 연결하자

`-` 이때 간선의 용량은 각 사람이 사려고 하는 책의 개수로 하면 되고 비용은 $0$이다

`-` 사람 노드마다 모든 서점 노드를 연결하자 (사람 $\to$ 서점)

`-` 이때 간선의 용량은 그 사람이 사려고 하는 책의 개수이고 비용은 배송비로 설정하면 된다

`-` 마지막으로 서점 노드와 싱크를 연결해주자

`-` 간선의 비용은 $0$이고 용량은 각 서점이 가지고 있는 책의 개수로 하면 된다

`-` 사람이 서점에서 책을 구매할 때만 배송비가 드니 사람과 서점을 잇는 간선 외에는 비용이 $0$이어야 한다

`-` 참고로 책 $1$권을 구매할 때마다 배송비를 지불해야 한다

`-` SPFA 구현할 때 큐에서 원소 꺼냈으면 꺼냈다고 마킹을 해야 되는데 빼먹어서 $2$트 컷 했다

`-` 로직을 생각하며 꼼꼼히 구현하자

In [1]:
from collections import defaultdict, deque


def ssp(graph, weight, source, sink, flow, capacity, num_nodes):
    max_flow = 0
    min_cost = 0
    exists_augmenting_path = True
    while exists_augmenting_path:
        distances, predecessors = spfa(graph, weight, source, flow, capacity, num_nodes)
        exists_augmenting_path = predecessors[sink] != NONE
        if not exists_augmenting_path:
            break
        augmenting_path = track_augmenting_path(predecessors, sink)
        bottleneck = compute_bottleneck(augmenting_path, flow, capacity)
        update_flow(augmenting_path, flow, bottleneck)
        cost = distances[sink]
        max_flow += bottleneck
        min_cost += bottleneck * cost
    return max_flow, min_cost


def spfa(graph, weight, source, flow, capacity, num_nodes):
    queue = deque([source])
    distances = [INF] * num_nodes
    distances[source] = 0
    predecessors = [NONE] * num_nodes
    in_queue = [False] * num_nodes
    in_queue[source] = True
    while queue:
        u = queue.popleft()
        in_queue[u] = False
        dist_u = distances[u]
        for v in graph[u]:
            dist_v = distances[v]
            w = weight[u][v]
            residual_capacity = capacity[u][v] - flow[u][v]
            if residual_capacity <= 0 or dist_v <= dist_u + w:
                continue
            distances[v] = dist_u + w
            predecessors[v] = u
            if in_queue[v]:
                continue
            in_queue[v] = True
            queue.append(v)
    return distances, predecessors


def track_augmenting_path(predecessors, sink):
    augmenting_path = []
    node = sink
    while node != NONE:
        augmenting_path.append(node)
        node = predecessors[node]
    augmenting_path.reverse()
    return augmenting_path


def compute_bottleneck(augmenting_path, flow, capacity):
    bottleneck = INF
    for i in range(len(augmenting_path) - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1]
        bottleneck = min(capacity[u][v] - flow[u][v], bottleneck)
    return bottleneck


def update_flow(augmenting_path, flow, bottleneck):
    for i in range(len(augmenting_path) - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1]
        flow[u][v] += bottleneck
        flow[v][u] -= bottleneck


def solution():
    global INF, NONE
    N, M = map(int, input().split())
    INF = float("inf")
    NONE = -1
    source = 0
    sink = N + M + 1
    num_nodes = N + M + 2
    graph = defaultdict(list)
    flow = [[0] * num_nodes for _ in range(num_nodes)]
    capacity = [[1] * num_nodes for _ in range(num_nodes)]
    weight = [[0] * num_nodes for _ in range(num_nodes)]
    person2num_books = list(map(int, input().split()))
    bookstore2num_books = list(map(int, input().split()))
    graph[source] = list(range(1, N + 1))
    for person in range(1, N + 1):
        capacity[source][person] = person2num_books[person - 1]
    for bookstore in range(N + 1, N + M + 1):
        graph[bookstore].append(sink)
        capacity[bookstore][sink] = bookstore2num_books[bookstore - N - 1]
    for bookstore in range(N + 1, N + M + 1):
        costs = list(map(int, input().split()))
        for person in range(1, N + 1):
            graph[person].append(bookstore)
            graph[bookstore].append(person)
            capacity[person][bookstore] = person2num_books[person - 1]
            capacity[bookstore][person] = 0
            weight[person][bookstore] = costs[person - 1]
            weight[bookstore][person] = -costs[person - 1]
    _, min_cost = ssp(graph, weight, source, sink, flow, capacity, num_nodes)
    print(min_cost)


solution()

# input
# 4 4
# 3 2 4 2
# 5 3 2 1
# 5 6 2 1
# 3 7 4 1
# 2 10 3 1
# 10 20 30 1

 4 4
 3 2 4 2
 5 3 2 1
 5 6 2 1
 3 7 4 1
 2 10 3 1
 10 20 30 1


30


## 책 구매하기 3

- 문제 출처: [백준 11407번](https://www.acmicpc.net/problem/11407)

`-` [책 구매하기](https://www.acmicpc.net/problem/11405) 문제에서 사람 $\to$ 서점 간선의 용량만 바뀐 문제이다

In [2]:
from collections import defaultdict, deque


def ssp(graph, weight, source, sink, flow, capacity, num_nodes):
    max_flow = 0
    min_cost = 0
    exists_augmenting_path = True
    while exists_augmenting_path:
        distances, predecessors = spfa(graph, weight, source, flow, capacity, num_nodes)
        exists_augmenting_path = predecessors[sink] != NONE
        if not exists_augmenting_path:
            break
        augmenting_path = track_augmenting_path(predecessors, sink)
        bottleneck = compute_bottleneck(augmenting_path, flow, capacity)
        update_flow(augmenting_path, flow, bottleneck)
        cost = distances[sink]
        max_flow += bottleneck
        min_cost += bottleneck * cost
    return max_flow, min_cost


def spfa(graph, weight, source, flow, capacity, num_nodes):
    queue = deque([source])
    distances = [INF] * num_nodes
    distances[source] = 0
    predecessors = [NONE] * num_nodes
    in_queue = [False] * num_nodes
    in_queue[source] = True
    while queue:
        u = queue.popleft()
        in_queue[u] = False
        dist_u = distances[u]
        for v in graph[u]:
            dist_v = distances[v]
            w = weight[u][v]
            residual_capacity = capacity[u][v] - flow[u][v]
            if residual_capacity <= 0 or dist_v <= dist_u + w:
                continue
            distances[v] = dist_u + w
            predecessors[v] = u
            if in_queue[v]:
                continue
            in_queue[v] = True
            queue.append(v)
    return distances, predecessors


def track_augmenting_path(predecessors, sink):
    augmenting_path = []
    node = sink
    while node != NONE:
        augmenting_path.append(node)
        node = predecessors[node]
    augmenting_path.reverse()
    return augmenting_path


def compute_bottleneck(augmenting_path, flow, capacity):
    bottleneck = INF
    for i in range(len(augmenting_path) - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1]
        bottleneck = min(capacity[u][v] - flow[u][v], bottleneck)
    return bottleneck


def update_flow(augmenting_path, flow, bottleneck):
    for i in range(len(augmenting_path) - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1]
        flow[u][v] += bottleneck
        flow[v][u] -= bottleneck


def solution():
    global INF, NONE
    N, M = map(int, input().split())
    INF = float("inf")
    NONE = -1
    source = 0
    sink = N + M + 1
    num_nodes = N + M + 2
    graph = defaultdict(list)
    flow = [[0] * num_nodes for _ in range(num_nodes)]
    capacity = [[1] * num_nodes for _ in range(num_nodes)]
    weight = [[0] * num_nodes for _ in range(num_nodes)]
    person2num_books = list(map(int, input().split()))
    bookstore2num_books = list(map(int, input().split()))
    graph[source] = list(range(1, N + 1))
    for person in range(1, N + 1):
        capacity[source][person] = person2num_books[person - 1]
    for bookstore in range(N + 1, N + M + 1):
        graph[bookstore].append(sink)
        capacity[bookstore][sink] = bookstore2num_books[bookstore - N - 1]
    for bookstore in range(N + 1, N + M + 1):
        limits = list(map(int, input().split()))
        for person in range(1, N + 1):
            graph[person].append(bookstore)
            graph[bookstore].append(person)
            capacity[person][bookstore] = limits[person - 1]
            capacity[bookstore][person] = 0
    for bookstore in range(N + 1, N + M + 1):
        costs = list(map(int, input().split()))
        for person in range(1, N + 1):
            weight[person][bookstore] = costs[person - 1]
            weight[bookstore][person] = -costs[person - 1]
    max_flow, min_cost = ssp(graph, weight, source, sink, flow, capacity, num_nodes)
    print(max_flow)
    print(min_cost)


solution()

# input
# 4 4
# 3 2 4 2
# 5 3 2 1
# 0 1 1 0
# 1 0 1 2
# 2 1 1 1
# 0 0 2 0
# 5 6 2 1
# 3 7 4 1
# 2 10 3 1
# 10 20 30 1

 4 4
 3 2 4 2
 5 3 2 1
 0 1 1 0
 1 0 1 2
 2 1 1 1
 0 0 2 0
 5 6 2 1
 3 7 4 1
 2 10 3 1
 10 20 30 1


8
47


## 두부장수 장홍준 2

- 문제 출처: [백준 11111번](https://www.acmicpc.net/problem/11111)

`-` 문제 번호가 $11111$이다!

`-` 의외로 MCMF인 문제라고 한다

`-` 격자에서 비트마스크 DP를 계산하는 문제인 [두부장수 장홍준](https://www.acmicpc.net/problem/1657)에 비해 두부판이 커졌다

`-` 둘 다 안 풀었는데 최소 비용 최대 유량 문제부터 풀어보자

`-` 생각해보니까 이 문제를 풀면 [두부장수 장홍준](https://www.acmicpc.net/problem/1657) 문제는 날먹이잖어?

`-` 두부판을 체스판이라 생각하고 흰색 칸이면 상하좌우 칸과 연결하자 (행과 열의 인덱스 합이 짝수)

`-` 각 칸은 한 번만 선택될 수 있으니 모든 간선의 용량은 $1$로 설정하자

`-` 소스 $\to$ 흰색 칸 $\to$ 검은색 칸 $\to$ 싱크와 같이 네트워크를 설계하면 된다

`-` 그럼 두부를 자를 수 있는 모든 경우를 문제없이 표현할 수 있다

`-` 이때 흰색 칸과 검은색 칸을 연결하는 간선의 가중치는 두부 가격에 $-1$을 곱한 값으로 설정하자 (나머지는 $0$)

`-` 이 그래프에서 최소 비용으로 최대 유량을 흘리면 최대 유량일 때 잘라낸 두부 가격의 합을 최대로 한다

`-` 문제는 최대 유량일 때 잘라낸 두부 가격의 합이 최적은 아니라는 것이다

`-` 예시를 보면 $8$의 유량을 흘려 $8$개의 두부를 만들 수 있지만 $7$개의 두부만 만들 때가 잘라낸 두부 가격의 합을 최대로 한다

`-` 잘 생각해보면 최소 비용으로 유량을 흘리므로 내가 원하는만큼 두부를 잘라내도 해당 개수를 기준으로 잘라낸 두부 가격의 합을 최대로 할 수 있다

`-` 예컨대 $1$의 유량을 흘리는 것은 두부판에서 $1$개의 두부를 잘라내는 것이다

`-` 이때 최소 비용으로 유량을 흘리므로 해당 두부는 두부판에서 가장 비쌀 것이다

`-` 두부판은 $N \times M$ 크기이므로 두부를 $0$개부터 $\left\lfloor\frac{NM}{2}\right\rfloor$개까지 잘라낼 수 있다

`-` 그리고 그 때마다 잘라낸 두부 가격의 합은 해당 개수만큼 잘라낸 기준으로 최대이다

`-` 따라서 모든 경우 중 두부 가격의 합이 최대일 때가 정답이다

`-` 유량을 흘리면서 최소 비용이 양수가 되면 멈추면 된다 (즉, 두부 $1$개를 추가로 잘라내서 얻은 이익이 음수이면 멈추자)

`-` 그래프의 정점은 $O(NM)$개이며 간선도 $O(NM)$개이다

`-` 최소 비용을 가지는 유량은 SPFA로 최악의 경우에도 $O\left(N^2 M^2\right)$에 찾을 수 있다

`-` 따라서 최소 비용 최대 유량의 시간 복잡도는 최악의 경우 $O\left(N^3 M^3\right)$이다

`-` $5$초 제한인데 $4608\operatorname{ms}$로 진짜 간신히 통과했다

`-` 의외로 MCMF인 문제라는 힌트가 큰 도움이 돼서 생각보다 쉽게 풀었다

`-` 최소 비용 최대 유량 문제를 오랜만에 풀었는데 제대로 구현해서 다행이다

`-` 꽤 재밌는 문제였다

In [35]:
from collections import defaultdict, deque
from itertools import product


def prepare_network(large_tofu, tofu2price, source, sink):
    n = len(large_tofu)
    m = len(large_tofu[0])
    size = sink + 1
    graph = [[] for _ in range(size)]
    flow = defaultdict(int)
    capacity = defaultdict(int)
    weight = defaultdict(int)
    drc = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    for r, c in product(range(n), range(m)):
        start = convert(r, c, m)
        if (r + c) % 2 == 1:
            graph[start].append(sink)
            capacity[start, sink] = 1
            continue
        graph[source].append(start)
        capacity[source, start] = 1
        for dr, dc in drc:
            nr = r + dr
            nc = c + dc
            is_in_range = 0 <= nr < n and 0 <= nc < m
            if not is_in_range:
                continue
            end = convert(nr, nc, m)
            graph[start].append(end)
            graph[end].append(start)
            capacity[start, end] = 1
            price = compute_price(large_tofu[r][c], large_tofu[nr][nc], tofu2price)
            weight[start, end] = -1 * price
            weight[end, start] = price
    return graph, flow, capacity, weight


def convert(r, c, col_size):
    return r * col_size + c + 1


def compute_price(cell_1, cell_2, tofu2price):
    tofu_1 = cell_1 + cell_2
    tofu_2 = cell_2 + cell_1
    return tofu2price.get(tofu_1, None) or tofu2price.get(tofu_2, None)


def ssp(graph, flow, capacity, weight, source, sink, size):
    max_price = 0
    exists_augmenting_path = True
    while exists_augmenting_path:
        distances, predecessors = spfa(graph, flow, capacity, weight, source, size)
        exists_augmenting_path = predecessors[sink] != None
        if not exists_augmenting_path:
            break
        price = -1 * distances[sink]
        if price <= 0:
            break
        max_price += price
        augmenting_path = track_augmenting_path(predecessors, sink)
        bottleneck = compute_bottleneck(flow, capacity, augmenting_path)
        update_flow(flow, augmenting_path, bottleneck)
    return max_price


def spfa(graph, flow, capacity, weight, source, size):
    queue = deque([source])
    distances = [INF] * size
    distances[source] = 0
    in_queue = [False] * size
    in_queue[source] = True
    predecessors = [None] * size
    while queue:
        u = queue.popleft()
        in_queue[u] = False
        d_u = distances[u]
        for v in graph[u]:
            d_v = distances[v]
            w = weight[u, v]
            residual_capacity = capacity[u, v] - flow[u, v]
            if residual_capacity <= 0 or d_v <= d_u + w:
                continue
            distances[v] = d_u + w
            predecessors[v] = u
            if in_queue[v]:
                continue
            queue.append(v)
            in_queue[v] = True
    return distances, predecessors


def track_augmenting_path(predecessors, sink):
    augmenting_path = []
    node = sink
    while node != None:
        augmenting_path.append(node)
        node = predecessors[node]
    augmenting_path = augmenting_path[::-1]
    return augmenting_path


def compute_bottleneck(flow, capacity, augmenting_path):
    bottleneck = INF
    n = len(augmenting_path)
    for i in range(n - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1]
        residual_capacity = capacity[u, v] - flow[u, v]
        bottleneck = min(residual_capacity, bottleneck)
    return bottleneck


def update_flow(flow, augmenting_path, bottleneck):
    n = len(augmenting_path)
    for i in range(n - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1]
        flow[u, v] += bottleneck
        flow[v, u] -= bottleneck


def solution():
    global INF
    N, M = map(int, input().split())
    large_tofu = [input() for _ in range(N)]
    tofu2price = {
        "AA": 10, "AB": 8, "AC": 7, "AD": 5, "AF": 1,
        "BB": 6, "BC": 4, "BD": 3, "BF": 1,
        "CC": 3, "CD": 2, "CF": 1,
        "DD": 2, "DF": 1,
        "FF": 0
    }
    INF = float("inf")
    source = 0
    sink = N * M + 1
    size = sink + 1
    graph, flow, capacity, weight = prepare_network(large_tofu, tofu2price, source, sink)
    max_price = ssp(graph, flow, capacity, weight, source, sink, size)
    print(max_price)


solution()

# input
# 4 4
# ACFC
# FDAB
# BACF
# DBAC

 4 4
 ACFC
 FDAB
 BACF
 DBAC


37


`-` [두부 모판 자르기](https://www.acmicpc.net/problem/10937) 문제 날먹하러 가자